In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Load the data
data1 = np.loadtxt('Marta_ML.txt', skiprows=2)
data2 = pd.read_csv('B2-2.txt', skiprows=4, delimiter='\s+', names=['Depth', 'Load', 'Time', 'Depth_V', 'Load_V'], encoding='ISO-8859-1')
# Extract columns for Dataset 1 (already in meters and newtons)
height1_m = data1[:, 1]  # Second column (in meters)
pressure1_n = data1[:, 2]  # Third column (in newtons)
# Extract and convert columns for Dataset 2
height2_m = data2['Depth'] * 1e-6  # Convert from nm to meters
pressure2_n = data2['Load'] * 1e-6  # Convert from µN to newtons
# Define target heights for special points (in meters)
target_heights = [0.4e-4, 0.8e-4, 1.2e-4]
# Find pressure values at the specified heights for Dataset 1
height1_points = []
pressure1_points = []
for target in target_heights:    
    closest_index = np.abs(height1_m - target).argmin()
    height1_points.append(height1_m[closest_index])    
    pressure1_points.append(pressure1_n[closest_index])
# Find pressure values at the specified heights for Dataset 2 (ensure pressures are > 0)
height2_points = []
pressure2_points = []
for target in target_heights:    
    valid_indices = (pressure2_n > 0) & (height2_m >= target)  # Ensure positive pressure and closest match
    if valid_indices.any():        
        closest_index = np.abs(height2_m[valid_indices] - target).idxmin()
        height2_points.append(height2_m.iloc[closest_index])        
        pressure2_points.append(pressure2_n.iloc[closest_index])
# Calculate RMSE for the target points
pressure1_points = np.array(pressure1_points)
pressure2_points = np.array(pressure2_points)
squared_differences = (pressure1_points - pressure2_points) ** 2
rmse_target_points = np.sqrt(np.mean(squared_differences))
# Find the peak and drop points for Dataset 1
peak_index1 = np.argmax(pressure1_n)  # Index of max pressure
peak_height1, peak_pressure1 = height1_m[peak_index1], pressure1_n[peak_index1]
drop_threshold1 = peak_pressure1 * 0.001  # Define a drop threshold (e.g., 90% drop from the peak)
drop_index1 = peak_index1 + np.where(pressure1_n[peak_index1:] <= drop_threshold1)[0][0]
drop_height1, drop_pressure1 = height1_m[drop_index1], pressure1_n[drop_index1]
# Find the peak and drop points for Dataset 2
peak_index2 = np.argmax(pressure2_n)  # Index of max pressure
peak_height2, peak_pressure2 = height2_m.iloc[peak_index2], pressure2_n.iloc[peak_index2]
drop_index2 = np.argmin(pressure2_n)  # Index of minimum pressure
drop_height2, drop_pressure2 = height2_m.iloc[drop_index2], pressure2_n.iloc[drop_index2]
# Calculate the error for peak and drop points
peak_error = np.abs(peak_pressure1 - peak_pressure2)
drop_error = np.abs(drop_pressure1 - drop_pressure2)
# Include the peak and drop pressures into the arrays for RMSE calculation
pressure1_all_points = np.append(pressure1_points, [peak_pressure1, drop_pressure1])
pressure2_all_points = np.append(pressure2_points, [peak_pressure2, drop_pressure2])
# Calculate RMSE for all 5 points
squared_differences_all = (pressure1_all_points - pressure2_all_points) ** 2
rmse_all_points = np.sqrt(np.mean(squared_differences_all))
# Calculate R^2 for all points
y_actual = pressure1_all_points
y_pred = pressure2_all_points
ss_res = np.sum((y_actual - y_pred)  ** 2)
ss_tot = np.sum((y_actual - np.mean(y_actual))  ** 2)
r_squared = 1 - (ss_res / ss_tot)
# Print RMSE, peak error, drop error, and R^2print(f"RMSE (target points): {rmse_target_points:.3e}")
print(f"Updated RMSE (5 points): {rmse_all_points:.3e}")
print(f"Peak Pressure Error: {peak_error:.3e}")
print(f"Drop Pressure Error: {drop_error:.3e}")
print(f"R^2 (5 points): {r_squared:.3f}")
# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(height1_m, pressure1_n, color='orange', linewidth=2, label='Dataset 1')
plt.plot(height2_m, pressure2_n, color='blue', linewidth=2, label='Dataset 2')
# Plot the specified height markers for Dataset 1
plt.scatter(height1_points, pressure1_points, color='red', marker='o', s=60, label='Target Heights (Dataset 1)')
# Plot the specified height markers for Dataset 2
plt.scatter(height2_points, pressure2_points, color='green', marker='o', s=60, label='Target Heights (Dataset 2)')
# Plot the peak and drop points for Dataset 1plt.scatter(peak_height1, peak_pressure1, color='red', marker='o', s=100, label='Peak (Dataset 1)')
plt.scatter(drop_height1, drop_pressure1, color='red', marker='o', s=100, label='Drop (Dataset 1)')
# Plot the peak and drop points for Dataset 2plt.scatter(peak_height2, peak_pressure2, color='green', marker='o', s=100, label='Peak (Dataset 2)')
plt.scatter(drop_height2, drop_pressure2, color='green', marker='o', s=100, label='Drop (Dataset 2)')
# Add parameters to the legend as an additional entryparameters_text = f"Parameters:\ntau0: 366.071429\na: 0.825\nh0: 10625\ntcs: 723.214286\nRMSE (5 pts): {rmse_all_points:.3e}\nPeak Error: {peak_error:.3e}\nDrop Error: {drop_error:.3e}\nR^2: {r_squared:.3f}"
plt.plot([], [], ' ', label=parameters_text)  # Empty plot to add text to the legend
# Add legendplt.legend(loc='upper left')
# Set labels and title with updated units
plt.xlabel('Height (m)')
plt.ylabel('Pressure (N)')
plt.title('Pressure vs Height for Two Datasets')
# Use scientific notation for y-axisplt.ticklabel_format(style='sci', axis='both', scilimits=(0, 0))
# Add grid
plt.grid(True, linestyle='--', alpha=0.7)
# Adjust layout to fit the elementsplt.tight_layout()
plt.show()